# Setup

In [21]:
import pandas as pd
import numpy as np
import os

# Program

## EDA

In [22]:
one_year_data = pd.read_csv("../data/final/oneyear_daily_data.csv")

In [23]:
one_year_data.describe()

,YEAR,MO,DY,T2M,WS2M,RH2M,PRECTOTCORR,LAT,LONG
count,383934.000000,383934.000000,383934.000000,383934.000000,383934.000000,383934.000000,383934.00000,383934.000000,383934.000000
mean,2023.754098,6.513661,15.756831,19.459908,-3.686209,71.776200,-4.51804,-8.520460,115.785572
std,0.430621,3.451238,8.811541,92.596452,90.505853,97.472976,90.79559,1.796311,14.009459
min,2023.000000,1.000000,1.000000,-999.000000,-999.000000,-999.000000,-999.00000,-11.162500,94.409500
25%,2024.000000,4.000000,8.000000,26.980000,2.910000,77.810000,0.18000,-10.162500,102.909500
50%,2024.000000,7.000000,16.000000,28.080000,4.590000,80.940000,0.94000,-8.662500,114.909500
75%,2024.000000,10.000000,23.000000,28.880000,6.090000,83.750000,3.83000,-7.162500,127.909500
max,2024.000000,12.000000,31.000000,32.290000,15.270000,99.000000,362.46000,-4.162500,140.909500


In [24]:
one_year_data =one_year_data[~((one_year_data["T2M"]==-999) | (one_year_data["WS2M"]==-999) | (one_year_data["RH2M"]==-999) | (one_year_data["PRECTOTCORR"]==-999))]

In [25]:
one_year_data.rename(columns={'YEAR':'Year', "MO":"Month", "DY":"Day", "HR":"Hour"}, inplace=True)

In [26]:
one_year_data

,Year,Month,Day,T2M,WS2M,RH2M,PRECTOTCORR,LAT,LONG
0,2023.0,10.0,3.0,25.96,8.09,75.50,0.14,-10.1625,94.4095
1,2023.0,10.0,4.0,26.10,7.73,73.25,0.23,-10.1625,94.4095
2,2023.0,10.0,5.0,25.94,6.56,81.00,2.06,-10.1625,94.4095
3,2023.0,10.0,6.0,26.16,5.86,82.75,0.71,-10.1625,94.4095
4,2023.0,10.0,7.0,26.22,5.58,83.38,0.83,-10.1625,94.4095
...,...,...,...,...,...,...,...,...,...
383929,2024.0,9.0,28.0,26.98,5.52,84.00,0.09,-9.6625,140.9095
383930,2024.0,9.0,29.0,26.96,6.51,84.12,0.05,-9.6625,140.9095
383931,2024.0,9.0,30.0,26.97,7.19,81.38,0.00,-9.6625,140.9095
383932,2024.0,10.0,1.0,26.73,7.19,80.12,0.16,-9.6625,140.9095


In [27]:
one_year_data["DATETIME"] = pd.to_datetime(one_year_data[["Year", "Month", "Day"]])

In [28]:
one_year_data.sort_values(by="DATETIME", inplace=True)

## Split Data

In [29]:
pivot = round(0.9*len(one_year_data))

In [30]:
train_data = one_year_data.iloc[:pivot]
test_data = one_year_data.iloc[pivot:]

## Modeling

In [31]:
formatted_data = one_year_data
formatted_data.set_index("DATETIME", inplace=True)

In [32]:
formatted_data[["T2M", "LAT", "LONG"]]

,T2M,LAT,LONG
DATETIME,,,
2023-10-03,25.96,-10.1625,94.4095
2023-10-03,26.10,-10.1625,108.4095
2023-10-03,26.76,-9.6625,127.4095
2023-10-03,26.45,-4.1625,94.9095
2023-10-03,26.12,-8.1625,140.9095
...,...,...,...
2024-10-02,28.77,-8.1625,117.4095
2024-10-02,28.00,-5.1625,94.4095
2024-10-02,28.18,-7.6625,102.9095


In [33]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

# Assuming df is your DataFrame with 'target', 'latitude', 'longitude', and a 'date' index

# Normalize the features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(formatted_data[['T2M', 'LAT', 'LONG']])


In [34]:
scaled_data[0,:]

array([0.56011119, 0.14285714, 0.        ])

In [35]:

# Create sequences
def create_sequences(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), :])
        y.append(data[i + time_step, 0])  # Target is the first column
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, time_step=1)

In [36]:
# Split into train and test
train_size = int(len(X) * 0.9)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [37]:

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer

model.compile(optimizer='adam', loss='mean_squared_error')

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [38]:
# Train the model
model.fit(X_train, y_train, epochs=1, batch_size=32)

10710/10710 ━━━━━━━━━━━━━━━━━━━━ 37s 3ms/step - loss: 0.0176


In [39]:
# Make predictions
predictions = model.predict(X_test)

1190/1190 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step


In [40]:
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae:.2f}')

Mean Absolute Error: 0.06


In [41]:
np.array([scaled_data[-10:,:]]).shape

(1, 10, 3)

In [42]:
prediction = model.predict(np.array([scaled_data[-1:,:]]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


In [43]:
scaler.inverse_transform(np.array([[prediction[0,0], None, None]]))

array([[27.49097406,         nan,         nan]])

In [44]:
import pickle

In [45]:
model.save("../model/temperature_forecast_model.h5")

In [46]:
from keras.models import load_model

In [47]:
loaded_model = load_model("../model/temperature_forecast_model.h5")

In [48]:
scaled_data[0,:]

array([0.56011119, 0.14285714, 0.        ])

In [49]:
loaded_model.predict(np.array([np.array([scaled_data[0,:]])]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 659ms/step


array([[0.6452574]], dtype=float32)

In [50]:
# Save the scaler to a file
with open('../model/temperature_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [51]:
import pickle

In [52]:
with open('../model/temperature_scaler.pkl', 'rb') as f:
    temperature_scaler = pickle.load(f)

In [53]:
temperature_scaler.fit_transform(formatted_data[['T2M', 'LAT', 'LONG']])

array([[0.56011119, 0.14285714, 0.        ],
       [0.56984017, 0.14285714, 0.30107527],
       [0.61570535, 0.21428571, 0.70967742],
       ...,
       [0.71438499, 0.5       , 0.1827957 ],
       [0.68033356, 0.42857143, 0.95698925],
       [0.63307853, 0.21428571, 1.        ]])

## Windspeed

In [54]:

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(formatted_data[['WS2M', 'LAT', 'LONG']])


In [55]:
X, y = create_sequences(scaled_data, time_step=1)

In [56]:
# Split into train and test
train_size = int(len(X) * 0.9)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [57]:

# Build the LSTM model
windspeed_model = Sequential()
windspeed_model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
windspeed_model.add(Dropout(0.2))
windspeed_model.add(LSTM(50, return_sequences=False))
windspeed_model.add(Dropout(0.2))
windspeed_model.add(Dense(1))  # Output layer

windspeed_model.compile(optimizer='adam', loss='mean_squared_error')

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [58]:
# Train the model
windspeed_model.fit(X_train, y_train, epochs=1, batch_size=32)

10710/10710 ━━━━━━━━━━━━━━━━━━━━ 36s 3ms/step - loss: 0.0178


In [59]:
# Make predictions
predictions = windspeed_model.predict(X_test)

1190/1190 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


In [60]:
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae:.2f}')

Mean Absolute Error: 0.10


In [61]:
windspeed_model.save("../model/windspeed_forecast_model.h5")

In [62]:
# Save the scaler to a file
with open('../model/windspeed_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

## Humidity

In [63]:

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(formatted_data[['RH2M', 'LAT', 'LONG']])

In [64]:
X, y = create_sequences(scaled_data, time_step=1)

In [65]:
# Split into train and test
train_size = int(len(X) * 0.9)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [66]:

# Build the LSTM model
humidity_model = Sequential()
humidity_model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
humidity_model.add(Dropout(0.2))
humidity_model.add(LSTM(50, return_sequences=False))
humidity_model.add(Dropout(0.2))
humidity_model.add(Dense(1))  # Output layer

humidity_model.compile(optimizer='adam', loss='mean_squared_error')

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [67]:
# Train the model
humidity_model.fit(X_train, y_train, epochs=1, batch_size=32)

10710/10710 ━━━━━━━━━━━━━━━━━━━━ 32s 3ms/step - loss: 0.0168


In [68]:
# Make predictions
predictions = humidity_model.predict(X_test)

1190/1190 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


In [69]:
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae:.2f}')

Mean Absolute Error: 0.07


In [70]:
humidity_model.save("../model/humidity_forecast_model.h5")

In [71]:
# Save the scaler to a file
with open('../model/humidity_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

## Precipitation

In [72]:

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(formatted_data[['PRECTOTCORR', 'LAT', 'LONG']])

In [73]:
X, y = create_sequences(scaled_data, time_step=10)

In [74]:
# Split into train and test
train_size = int(len(X) * 0.9)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [75]:

# Build the LSTM model
precipitation_model = Sequential()
precipitation_model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
precipitation_model.add(Dropout(0.2))
precipitation_model.add(LSTM(50, return_sequences=False))
precipitation_model.add(Dropout(0.2))
precipitation_model.add(Dense(1))  # Output layer

precipitation_model.compile(optimizer='adam', loss='mean_squared_error')

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [76]:
# Train the model
precipitation_model.fit(X_train, y_train, epochs=1, batch_size=32)

10710/10710 ━━━━━━━━━━━━━━━━━━━━ 104s 9ms/step - loss: 5.6937e-04


In [77]:
# Make predictions
predictions = precipitation_model.predict(X_test)

1190/1190 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step


In [78]:
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae:.2f}')

Mean Absolute Error: 0.00


In [79]:
precipitation_model.save("../model/precipitation_forecast_model.h5")

In [80]:
# Save the scaler to a file
with open('../model/precipitation_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)